In [1]:
# -------------------------- Python & library version --------------------------
# Python version: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
# pandas version: 1.2.4
# numpy version: 1.19.2
# matplotlib version: 3.3.4
# tqdm version: 4.59.0
# sktime version: 0.6.1
# xgboost version: 1.2.1
# seaborn version: 0.11.1
# scikit-learn version: 0.24.2
# ------------------------------------------------------------------------------

In [2]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
# import torch
# import torch.nn as nn
# import lightgbm as lgb
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
pandas version: 2.1.1
numpy version: 1.26.0
matplotlib version: 3.8.0
tqdm version: 4.66.1
sktime version: 0.24.0
xgboost version: 1.7.3
seaborn version: 0.13.0
scikit-learn version: 1.2.2
------------------------------------------------------------------------------


In [8]:
import os, contextlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler


import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 30)

In [128]:
train = pd.read_csv('data/train_28-28_preproc_5.csv')

In [111]:
train

,item,supply,price,price_t-56,price_t-55,price_t-54,price_t-53,price_t-52,price_t-51,price_t-50,price_t-49,price_t-48,price_t-47,price_t-46,price_t-45,...,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
0,TG,4057.0,2249.0,0.0,0.0,1728.0,1408.0,1250.0,0.0,1474.0,1326.0,1428.0,1433.0,1546.0,1374.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,TG,3800.0,1672.0,0.0,1728.0,1408.0,1250.0,0.0,1474.0,1326.0,1428.0,1433.0,1546.0,1374.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,TG,5540.0,3841.0,1728.0,1408.0,1250.0,0.0,1474.0,1326.0,1428.0,1433.0,1546.0,1374.0,0.0,1694.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,TG,1700.0,4307.0,1408.0,1250.0,0.0,1474.0,1326.0,1428.0,1433.0,1546.0,1374.0,0.0,1694.0,1456.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,TG,1967.0,4538.0,1250.0,0.0,1474.0,1326.0,1428.0,1433.0,1546.0,1374.0,0.0,1694.0,1456.0,1470.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56116,BC,3192.0,2214.0,1470.0,0.0,1375.0,1206.0,1905.0,2500.0,0.0,2341.0,2388.0,2708.0,2186.0,2086.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
56117,BC,1968.0,2190.0,0.0,1375.0,1206.0,1905.0,2500.0,0.0,2341.0,2388.0,2708.0,2186.0,2086.0,1548.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
56118,BC,1824.0,2224.0,1375.0,1206.0,1905.0,2500.0,0.0,2341.0,2388.0,2708.0,2186.0,2086.0,1548.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
56119,BC,2576.0,2631.0,1206.0,1905.0,2500.0,0.0,2341.0,2388.0,2708.0,2186.0,2086.0,1548.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [117]:
train.columns

Index(['item', 'supply', 'price', 'price_t-56', 'price_t-55', 'price_t-54',
       'price_t-53', 'price_t-52', 'price_t-51', 'price_t-50',
       ...
       'month_10', 'month_11', 'month_12', 'dow_0', 'dow_1', 'dow_2', 'dow_3',
       'dow_4', 'dow_5', 'dow_6'],
      dtype='object', length=116)

In [12]:
## gridsearchCV for best model : 대략 1시간 소요

from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

df = pd.DataFrame(columns = ['n_estimators', 'min_child_weight','max_depth', 'colsample_bytree', 'subsample', 'alpha'])
preds = np.array([])

target_cols = [f'price_t + {i}' for i in range(28)]

grid = {
    'n_estimators' : [1000], 'eta' : [0.01],
	# 'min_child_weight' : np.arange(3, 8, 1),
    # 'max_depth' : np.arange(3,8,1) ,
	# 'colsample_bytree' :np.arange(0.4, 1.0, 0.2),
    # 'subsample' :np.arange(0.5, 1.0, 0.1),
}


for i in tqdm(train.item.unique()):
    y = train.loc[train.item == i, target_cols]
    x = train.loc[train.item == i, ].drop(['item', 'supply', 'price'] + target_cols, axis=1)
    test_size = int(len(y) * 0.1)
    y_train, y_test, x_train, x_test = temporal_train_test_split(y = y, X = x, test_size = test_size)
    
    pds = PredefinedSplit(np.append(-np.ones(len(x_train)-test_size), np.zeros(test_size)))
    gcv = GridSearchCV(estimator = XGBRegressor(seed = 0,
                                                gpu_id = 0, 
                                                # tree_method = 'gpu_hist',
                                                # predictor= 'gpu_predictor',
                                                verbose = 2
                                                ),
                        param_grid = grid,
                    #    scoring = mean_squared_error,
                        cv=pds,
                        refit = True,
                        verbose = 2,
                        n_jobs = -1)
    

    gcv.fit(x_train, y_train,
            early_stopping_rounds=10,
            eval_set=[(x_train, y_train), (x_test, y_test)],)
    best = gcv.best_estimator_
    params = gcv.best_params_

    pred = best.predict(x_test)
        
    # preds = np.append(preds, pred)
    preds = np.append(pred, preds)
    # df = pd.concat([df, pd.DataFrame(params, index = [0])], axis = 0)
    df = pd.concat([pd.DataFrame(params, index = [0]), df], axis = 0)

df.to_csv('./hyperparameter.csv', index = False) # save the tuned parameters


  0%|          | 0/5 [00:00<?, ?it/s]

Fitting 1 folds for each of 1 candidates, totalling 1 fits


KeyboardInterrupt: 

In [7]:
xgb_params = pd.read_csv('./hyperparameter.csv')

In [57]:
xgb_params.to_csv('./hyperparameter_xgb_final.csv', index=False)

In [ ]:
## best hyperparameters 불러오기
# xgb_params = pd.read_csv('smsle/hyperparameter_teacher.csv')
# xgb_params.head()

In [130]:
test = pd.read_csv('./data/test_28_preproc_5.csv')
test['price'] = 0
models = []
preds = {}

for i in tqdm(train.item.unique()):
    pred_df = pd.DataFrame()
    y = train.loc[train.item == i, target_cols]
    x = train.loc[train.item == i, ].drop(['item', 'supply', 'price'] + target_cols, axis=1)
    x_test = test.loc[test.item == i, x.columns]
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x)

    for seed in range(5): # 각 시드별 예측
        reg = XGBRegressor(seed = seed,
                        n_estimators = 1000,
                        # eta = 0.01,
                        # min_child_weight = params['min_child_weight'],
                        # max_depth = int(params['max_depth']),
                        # colsample_bytree = params['colsample_bytree'],
                        # subsample = params['subsample'],
                        # reg_alpha = params['reg_alpha'],
                        # reg_lambda = params['reg_lambda'],
                        )
        
        reg.fit(x_train, y)
        y_pred = reg.predict(scaler.transform(x_test))
        tmp = []
        for p in y_pred:
            tmp += list(p)
        pred_df.loc[:,seed] = tmp   # 각 시드별 예측 담기

    pred = pred_df.mean(axis=1)        # (i+1)번째 건물의 예측 =  (i+1)번째 건물의 각 시드별 예측 평균값
    test.loc[test.item == i, 'price'] = pred
    preds[i] = pred
    models.append(reg)

  0%|          | 0/5 [00:00<?, ?it/s]

In [123]:
test

,item,price_t-56,price_t-55,price_t-54,price_t-53,price_t-52,price_t-51,price_t-50,price_t-49,price_t-48,price_t-47,price_t-46,price_t-45,price_t-44,price_t-43,...,month_6,month_7,month_8,month_9,month_10,month_11,month_12,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,price
0,TG,1834.0,0.0,1780.0,1712.0,1911.0,2350.0,2819.0,1947.0,0.0,2079.0,2257.0,2507.0,2327.0,1503.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-111.589943
1,TG,2475.0,0.0,2466.0,2413.0,2777.0,2429.0,2669.0,2028.0,0.0,2304.0,2613.0,2466.0,2573.0,2243.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2696.145020
2,TG,1484.0,0.0,1625.0,1657.0,1983.0,2164.0,2178.0,2567.0,0.0,1861.0,1458.0,1435.0,1780.0,1310.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3404.476562
3,TG,1918.0,0.0,2390.0,2375.0,2798.0,3074.0,2866.0,2958.0,0.0,2821.0,2634.0,2370.0,2400.0,2176.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3658.817627
4,TG,2074.0,0.0,2027.0,2067.0,0.0,2079.0,2381.0,2162.0,0.0,2019.0,1987.0,1791.0,2155.0,1997.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4045.821777
5,TG,2351.0,0.0,2673.0,2517.0,0.0,2953.0,2934.0,2487.0,0.0,2097.0,2317.0,2871.0,3356.0,2830.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2564.540283
6,TG,2169.0,0.0,2148.0,1493.0,1172.0,2466.0,3981.0,3504.0,0.0,3176.0,658.0,1178.0,1244.0,957.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3871.020264
7,TG,2510.0,0.0,3448.0,4073.0,3667.0,3886.0,4448.0,3645.0,0.0,3167.0,3294.0,3217.0,2655.0,2804.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-129.629745
8,TG,1878.0,0.0,2295.0,1898.0,2352.0,2127.0,2020.0,1723.0,0.0,2162.0,2354.0,2384.0,2363.0,2467.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3518.055420
9,TG,2030.0,0.0,2729.0,3098.0,2605.0,2431.0,2756.0,3064.0,0.0,3443.0,2665.0,2640.0,2687.0,2558.0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3026.297363


In [121]:
raw_test = pd.read_csv('data/test.csv')
len(raw_test)

1092

In [131]:
raw_test['price'] = 0
for i in tqdm(test.item.unique()):
    print(len(raw_test.loc[raw_test.item == i, 'price']), i)
    raw_test.loc[raw_test.item == i, 'price'] = preds[i].values
raw_test

  0%|          | 0/5 [00:00<?, ?it/s]

280 TG
196 CR
140 CB
224 RD
252 BC


,ID,timestamp,item,corporation,location,price
0,TG_A_J_20230304,2023-03-04,TG,A,J,-678.679443
1,TG_A_J_20230305,2023-03-05,TG,A,J,3065.040283
2,TG_A_J_20230306,2023-03-06,TG,A,J,2863.672119
3,TG_A_J_20230307,2023-03-07,TG,A,J,2325.906006
4,TG_A_J_20230308,2023-03-08,TG,A,J,3826.553467
...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,532.706848
1088,RD_F_J_20230328,2023-03-28,RD,F,J,449.588074
1089,RD_F_J_20230329,2023-03-29,RD,F,J,382.600739
1090,RD_F_J_20230330,2023-03-30,RD,F,J,508.188416


In [132]:
raw_test.loc[raw_test.price < 0, 'price'] = 0

In [133]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['answer'] = raw_test.price
submission.to_csv('./data/submission/submission_28-28_preproc5_xgb.csv', index = False)